In [1]:
import numpy as np
import scipy.optimize as sp
import QuantLib as ql

In [2]:
import yfinance as yf
import datetime

start_date = datetime.datetime.now() - datetime.timedelta(days=2*365)
end_date = datetime.datetime.now()

sp500_data = yf.download('^GSPC', start=start_date, end=end_date)
vix_data = yf.download('^VIX', start=start_date, end=end_date)
common_dates = sp500_data.index.intersection(vix_data.index)
sp500 = sp500_data.loc[common_dates]['Close'].values
vix = vix_data.loc[common_dates]['Close'].values
kappa=2.5
delta=kappa/len(sp500)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
# interest rate
treasury_data = yf.download('^IRX', period="2y")
irx = treasury_data['Close'] / 100 
r = np.log(1 + irx).iloc[-1]

[*********************100%%**********************]  1 of 1 completed


In [5]:
vt=(vix/100)**2
#eatimates theta
def Xt(t,theta):
    return (vt[t+1] - vt[t] - kappa * (theta - vt[t]) * delta) / np.sqrt(delta * vt[t])
def avgXt(theta):
    return np.mean([Xt(t, theta) for t in range(len(vix)-1)])
theta_hat=sp.newton(lambda theta:avgXt(theta),10)

In [5]:
def Xt_simulation():
    simulations = []
    for _ in range(len(vt)-1):
        Xt=(vt[_+1] - vt[_] - kappa * (theta_hat - vt[_]) * delta) / np.sqrt(delta * vt[_])
        simulations.append(Xt)
    return simulations
Xt_simulation=Xt_simulation()
volvol=np.var(Xt_simulation)

In [6]:
#estimates mu
St=sp500
def Yt(t,mu):
    return (St[t+1] - St[t] - St[t] * mu * delta) / (St[t] * np.sqrt(delta * vt[t]))
def avgYt(mu):
    return np.mean([Yt(t, mu) for t in t_values])
mu_hat=sp.newton(lambda mu:avgYt(mu),10)

In [7]:
def Yt_simulation():
    simulations = []
    for _ in range(len(St)-1):
        Yt=(St[_+1] - St[_] - St[_] * mu_hat * delta) / (St[_] * np.sqrt(delta * vt[_]))
        simulations.append(Yt)
    return simulations
Yt_simulation=Yt_simulation()

In [8]:
#estimates rho
rho_hat=np.corrcoef(Xt_simulation,Yt_simulation)[0, 1]
rho_hat, theta_hat,volvol,mu_hat

(-0.7182839958091324,
 0.044313263271683834,
 0.13498477783275173,
 0.07162271190052115)

In [9]:
today = ql.Date(datetime.date.today().day, datetime.date.today().month, datetime.date.today().year)
day_count=ql.Actual365Fixed()
maturity = today + ql.Period(1, ql.Years)

S0=sp500[-1]
v0=vt[-1]
strike= S0
div=0

initial_value=ql.QuoteHandle(ql.SimpleQuote(S0))
discount_curves=ql.YieldTermStructureHandle(ql.FlatForward(today,r,day_count))
divident_curves=ql.YieldTermStructureHandle(ql.FlatForward(today,div,day_count))
heston_process=ql.HestonProcess(discount_curves,divident_curves,initial_value,v0,kappa,theta_hat,volvol,rho_hat)
call_payoff=ql.PlainVanillaPayoff(ql.Option.Call,strike)
call_exercise=ql.EuropeanExercise(maturity)

option = ql.VanillaOption(call_payoff, call_exercise)
engine = ql.AnalyticHestonEngine(ql.HestonModel(heston_process), 0.0001, 1000)
option.setPricingEngine(engine)
price = option.NPV()


print(f"Option Price:{price}")

Option Price:530.4560396277269


In [10]:
def calculate_delta(option, underlying, volatility):
    epsilon = 0.001
    underlying_value = underlying.value()
    
    underlying.setValue(underlying_value * (1 + epsilon))
    price_up = option.NPV()
    
    underlying.setValue(underlying_value * (1 - epsilon))
    price_down = option.NPV()
    
    underlying.setValue(underlying_value)
    
    delta = (price_up - price_down) / (2 * underlying_value * epsilon)
    return delta

def calculate_vega(option, volatility):
    epsilon = 0.001
    volatility_value = volatility.value()
    
    volatility.setValue(volatility_value + epsilon)
    price_up = option.NPV()
    
    volatility.setValue(volatility_value - epsilon)
    price_down = option.NPV()

    volatility.setValue(volatility_value)
    
    vega = (price_up - price_down) / (2 * epsilon)
    return vega

# create option
maturity_date = maturity
spot_price = S0
strike_price = S0
initial_volatility = v0
option_type = ql.Option.Call
risk_free_rate = r
dividend_rate = 0
mean_reversion = theta_hat
volatility = volvol
correlation = rho_hat

option = ql.VanillaOption(ql.PlainVanillaPayoff(option_type, strike_price), ql.EuropeanExercise(maturity_date))

# set heston model 
day_count = ql.Actual365Fixed()
calculation_date = ql.Date(1, 1, 2024)
ql.Settings.instance().evaluationDate = calculation_date

underlying = ql.SimpleQuote(spot_price)
volatility_quote = ql.SimpleQuote(initial_volatility)
risk_free_curve = ql.FlatForward(calculation_date, risk_free_rate, day_count)
dividend_curve = ql.FlatForward(calculation_date, dividend_rate, day_count)
heston_process = ql.HestonProcess(ql.YieldTermStructureHandle(dividend_curve), ql.YieldTermStructureHandle(risk_free_curve), ql.QuoteHandle(underlying), initial_volatility, volatility, mean_reversion, volatility * volatility, correlation)

option.setPricingEngine(ql.AnalyticHestonEngine(ql.HestonModel(heston_process)))

# calculate Delta and Vega
delta = calculate_delta(option, underlying, volatility_quote)
vega = calculate_vega(option, volatility_quote)

print(f"Delta: {delta}, Vega: {vega}")

Delta: 0.3957502784424764, Vega: 0.0


In [11]:
def generate_stock_path(S0, maturity, r, kappa, theta, volvol, rho, n_steps, n_simulations, dt):
    np.random.seed(2024)
    Z1 = np.random.normal(size=(n_steps, n_simulations))
    Z2 = rho * Z1 + np.sqrt(1 - rho ** 2) * np.random.normal(size=(n_steps, n_simulations))
    
    S = np.zeros((n_steps + 1, n_simulations))
    v = np.zeros((n_steps + 1, n_simulations))
    S[0] = S0
    v[0] = theta
    #print(S0, maturity, r, kappa, theta, volvol, rho, n_steps, n_simulations, dt)
    
    for i in range(1, n_steps + 1):
        v[i] = np.maximum(v[i - 1] + kappa * (theta - v[i - 1]) * dt + volvol * np.sqrt(v[i - 1] * dt) * Z1[i - 1], 0)
        S[i] = S[i - 1] * np.exp((r - 0.5 * v[i - 1]) * dt + np.sqrt(v[i - 1] * dt) * Z2[i - 1])
    
    return S, v

In [12]:
def monte_carlo_simulation(S0, strike, maturity, r, kappa, theta, volvol, rho, n_steps, n_simulations):
    dt = maturity / n_steps
    
    S, v = generate_stock_path(S0, maturity, r, kappa, theta, volvol, rho, n_steps, n_simulations, dt)
    payoffs = np.maximum(S[-1] - strike, 0)
    option_price = np.exp(-r * maturity) * payoffs
    
    return np.mean(option_price)

In [13]:
option_price = monte_carlo_simulation(S0=S0, strike=strike, r=r, kappa=kappa, theta=theta_hat,volvol=volvol, rho=rho_hat, maturity=1, n_steps=250, n_simulations=10000)

In [14]:
option_price,S0,r,v0,volvol,strike

(555.0710891243482,
 5022.2099609375,
 0.05110177633905688,
 0.0331604066656495,
 0.13498477783275173,
 5022.2099609375)

In [16]:
actual_price_list = [524.72, 493.10, 456.61, 447.42, 412.00]
actual_strike_list = [4975, 5000, 5025, 5050, 5100]

In [26]:
def objective_function(params):
    kappa, theta = params  
    total_error = 0
    for i in range(len(actual_price_list)):
        option_price = monte_carlo_simulation(S0=S0, strike=actual_strike_list[i], r=r, kappa=kappa, theta=theta, volvol=volvol, rho=rho_hat, maturity=1, n_steps=250, n_simulations=10000)
        error = (option_price - actual_price_list[i])**2
        total_error += error
        print(f"For strike price {actual_strike_list[i]}, Option price: {option_price}, Actual price: {actual_price_list[i]}, Error: {error}")
    print(f"Kappa:{kappa},Theta:{theta},Total error: {total_error}")
    return total_error

initial_guess = [2.5, 0.044]
bounds = [(1, None), (0, None)]
result = sp.minimize(objective_function, initial_guess, bounds=bounds, method='Nelder-Mead',options={'maxiter': 1000})

best_params = result.x
best_error = result.fun

print("Best parameters:", best_params)
print("Best error:", best_error)

For strike price 4975, Option price: 579.9487122505262, Actual price: 524.72, Error: 3050.2106568514228
For strike price 5000, Option price: 565.920444323652, Actual price: 493.1, Error: 5302.817111494092
For strike price 5025, Option price: 552.132705391085, Actual price: 456.61, Error: 9124.587245232013
For strike price 5050, Option price: 538.5564524060773, Actual price: 447.42, Error: 8305.852957165182
For strike price 5100, Option price: 511.9758763582225, Actual price: 412.0, Error: 9995.175853594588
Kappa:2.5,Theta:0.044,Total error: 35778.6438243373
For strike price 4975, Option price: 579.9345028977199, Actual price: 524.72, Error: 3048.6413302423143
For strike price 5000, Option price: 565.9232153832231, Actual price: 493.1, Error: 5303.220698751302
For strike price 5025, Option price: 552.1525730363497, Actual price: 456.61, Error: 9128.383262406222
For strike price 5050, Option price: 538.5883132006076, Actual price: 447.42, Error: 8311.661331844072
For strike price 5100, O

In [27]:
kappa, theta = 0, 0.02734852
total_error = 0
for i in range(len(actual_price_list)):
    option_price = monte_carlo_simulation(S0=S0, strike=actual_strike_list[i], r=r, kappa=kappa, theta=theta, volvol=volvol, rho=rho_hat, maturity=1, n_steps=250, n_simulations=10000)
    error = (option_price - actual_price_list[i])**2
    total_error += error
    print(f"For strike price {actual_strike_list[i]}, Option price: {option_price}, Actual price: {actual_price_list[i]}, Error: {error}")
print(f"Kappa:{kappa},Theta:{theta},Total error: {total_error}")

For strike price 4975, Option price: 499.56102530464045, Actual price: 524.72, Error: 632.9740077217434
For strike price 5000, Option price: 483.7728948224795, Actual price: 493.1, Error: 86.9948909925304
For strike price 5025, Option price: 468.27073627763855, Actual price: 456.61, Error: 135.97277053663538
For strike price 5050, Option price: 453.0343193164271, Actual price: 447.42, Error: 31.520581386806455
For strike price 5100, Option price: 423.34782184257404, Actual price: 412.0, Error: 128.77306057080037
Kappa:0,Theta:0.02734852,Total error: 1016.235311208516


In [24]:
r

0.05097824494588458